In [1]:
from pyspark.sql import *
spark=SparkSession.builder.appName('s27').getOrCreate()

In [2]:
df=spark.read.csv('/FileStore/tables/provider_level_measure_rates_for_ahrq_patient_safety_indicator_11_psi_11-9fa94.csv', inferSchema=True, header=True)

In [3]:
df1=spark.read.csv('/FileStore/tables/provider_level_measure_rates_for_ahrq_patient_safety_indicator_11_psi_11_2016-a3156.csv', inferSchema=True, header=True)

In [4]:
df.createOrReplaceTempView('data')
df1.createOrReplaceTempView('data1')

In [5]:
df2=spark.sql('select hosp_id from data1 where exists(select start_date from data where data.rate=data1.rate)')

In [6]:
df2.show(5)

+-------+
hosp_id|
+-------+
 10001|
 10005|
 10007|
 10011|
 10012|
+-------+
only showing top 5 rows

In [7]:
df3=spark.sql('select end_date from data where not exists(select hosp_id from data1 where data1.hosp_id=data.hosp_id)') # here we are fetching the data of information like is there any one hosp_id from data1 which hasn't been atleast one time reported again

In [8]:
df3.show(5)

+-------------------+
 end_date|
+-------------------+
2013-06-30 00:00:00|
2013-06-30 00:00:00|
2013-06-30 00:00:00|
2013-06-30 00:00:00|
2013-06-30 00:00:00|
+-------------------+
only showing top 5 rows

In [9]:
df4=spark.sql('select hosp_id from data where not exists(select end_date from data1 where data1.hosp_id=data.hosp_id)')

In [10]:
df4.show(5)

+-------+
hosp_id|
+-------+
 10009|
 10010|
 10015|
 10025|
 10027|
+-------+
only showing top 5 rows

In [11]:
df.dropDuplicates(['hosp_id']).select('hosp_id').show(10)

+-------+
hosp_id|
+-------+
 40011|
 50348|
 50353|
 80006|
 130003|
 150051|
 150181|
 170010|
 260195|
 340002|
+-------+
only showing top 10 rows

In [12]:
df5=spark.sql('select adm_disc from data where exists(select hosp_id from data1 where data.adm_disc=data1.adm_disc)')

In [13]:
df5.show(5)

+--------+
adm_disc|
+--------+
 269|
 56|
 48|
 1026|
 196|
+--------+
only showing top 5 rows

In [14]:
df6=spark.read.csv('/FileStore/tables/company.txt', inferSchema=True, header=True)

In [15]:
df6.show()

+---------+-----------+--------------+---------------+
companyid|companyname|companyproduct|customeraddress|
+---------+-----------+--------------+---------------+
 1| samsung| cellphone| tarnaka|
 2| nokia| tv| secunderabad|
 3| reliance| freeze| vidyanagar|
 4| jio|washingmachine| warasiguda|
 5| idea| sim| basthi|
+---------+-----------+--------------+---------------+

In [16]:
df7=spark.read.csv('/FileStore/tables/customer.txt', inferSchema=True, header=True)

In [17]:
df7.show()

+----------+------------+---------------+---------------+
customerid|customername|customerproduct|customeraddress|
+----------+------------+---------------+---------------+
 1| samsung| cellphone| tarnaka|
 2| nokia| tv| secunderabad|
 3| reliance| freeze| thirumalagiri|
 4| jio| washingmachine| warasiguda|
 5| idea| sim| basthi|
+----------+------------+---------------+---------------+

In [18]:
df6.createOrReplaceTempView('data2')
df7.createOrReplaceTempView('data3')

In [19]:
df8=spark.sql('select * from data3 d3 where exists(select customeraddress from data2 d2 where d2.customeraddress=d3.customeraddress )')

In [20]:
df8.show()

+----------+------------+---------------+---------------+
customerid|customername|customerproduct|customeraddress|
+----------+------------+---------------+---------------+
 1| samsung| cellphone| tarnaka|
 2| nokia| tv| secunderabad|
 4| jio| washingmachine| warasiguda|
 5| idea| sim| basthi|
+----------+------------+---------------+---------------+

In [21]:
df9=spark.sql('select * from data3 d3 where not exists(select customeraddress from data2 d2 where d2.customeraddress=d3.customeraddress )')

In [22]:
df9.show()

+----------+------------+---------------+---------------+
customerid|customername|customerproduct|customeraddress|
+----------+------------+---------------+---------------+
 3| reliance| freeze| thirumalagiri|
+----------+------------+---------------+---------------+

In [23]:
df10=spark.sql('select customerid from data3 where customeraddress = (select customeraddress from data2 where companyid=1)')

In [24]:
df10.show()

+----------+
customerid|
+----------+
 2|
+----------+

In [25]:
df11=spark.sql('select companyname from data2 where companyid >= 1')

In [26]:
df11.show()

+-----------+
companyname|
+-----------+
 samsung|
 nokia|
 reliance|
 jio|
 idea|
+-----------+

In [27]:
df12=spark.sql('select hosp_id from data where adm_disc in(select adm_disc from data1 where adm_disc=hosp_id)')

In [28]:
df12.show()

+-------+
hosp_id|
+-------+
+-------+

In [29]:
df13=spark.sql('select hosp_id from data where hosp_id in(select hosp_id from data1 where hosp_id>10005)')


In [30]:
df13.show(5)

+-------+
hosp_id|
+-------+
 10006|
 10007|
 10008|
 10011|
 10012|
+-------+
only showing top 5 rows

In [31]:
df14=spark.sql('select (select count(*) from data where adm_disc>80)*100/count(*) as value from data1')

In [32]:
df14.show()

+-----------------+
 value|
+-----------------+
76.19764989454654|
+-----------------+

In [33]:
df15=spark.sql('select start_date from data where hosp_id in(select hosp_id from data1 where rate>50)')

In [34]:
df15.show()

+----------+
start_date|
+----------+
+----------+

In [35]:
df16=spark.sql('select customerid from data3 d3 where exists(select companyname from data2 d2 where d2.customeraddress=d3.customeraddress )')

In [36]:
df16.show()

+----------+
customerid|
+----------+
 1|
 2|
 4|
 5|
+----------+

In [37]:
df17=spark.sql('select max(hosp_id) as max,min(rate) as min,avg(adm_disc) as avg,count(*) as count from data')

In [38]:
df17.show()

+------+----+-----------------+-----+
 max| min| avg|count|
+------+----+-----------------+-----+
670085|3.49|745.5636110142907| 3410|
+------+----+-----------------+-----+